# AI Sign Language Interpreter - CNN Model

## 1. Imports

In [27]:
# Notebook: CNN Deep Learning Model

import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

## 2. Data Preprocessing for Modeling

This section loads and normalizes the sign language data to prepare it for training and evaluation.

In [28]:
X_train = pd.read_csv('../data/processed/X_train_scaled.csv')
X_test = pd.read_csv('../data/processed/X_test_scaled.csv')
Y_train = pd.read_csv('../data/processed/y_train.csv')
Y_test = pd.read_csv('../data/processed/y_test.csv')

Y_train

,label
0,3
1,6
2,2
3,2
4,13
...,...
27450,13
27451,23
27452,18
27453,17


In [ ]:
# Reshape images to (28, 28, 1) for modeling. 
# The array is converted from 1-D to 3-D which is the required input to the first layer of the CNN.

X_train = X_train.to_numpy().reshape(-1, 28, 28, 1)
X_test = X_test.to_numpy().reshape(-1, 28, 28, 1)

In [30]:
# Fix encoding for categorical_crossentropy

Y_train = to_categorical(Y_train, num_classes=25)
Y_test = to_categorical(Y_test, num_classes=25)

print(Y_train.shape)
print(Y_test.shape)

(27455, 25)
(7172, 25)


## 3. Creating and Layering CNN Model

### CNN Model Overview

- **1st Convolutional Layer**: 128 filters, 5×5 kernel size, ReLU activation
- **1st Max Pooling Layer**: 3×3 pool size, stride 2

- **2nd Convolutional Layer**: 64 filters, 2×2 kernel size, ReLU activation
- **2nd Max Pooling Layer**: 2×2 pool size, stride 2  

- **3rd Convolutional Layer**: 32 filters, 2×2 kernel size, ReLU activation 
- **3rd Max Pooling Layer**: 2×2 pool size, stride 2

In [31]:
model = Sequential()

In [32]:
# Adding the first convolutional and pooling layer

model.add(Conv2D(128, kernel_size=(5,5), strides=1, padding='same', 
                    activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(3,3), strides=2, padding='same'))

/Users/adityasharma/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
# Adding the second convolutional and pooling layer

model.add(Conv2D(64, kernel_size=(2,2), strides=1, activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='same'))

In [34]:
# Adding the third convolutional and pooling layer

model.add(Conv2D(32, kernel_size=(2,2), strides=1, activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='same'))

In [35]:
model.add(Flatten())  # turn 3D into 1D (rolling image into list)

**Dense and Output Layers**

In [36]:
model.add(Dense(512, activation='relu'))  # fully connected layer to learn combinations of features
model.add(Dropout(0.25))  # prevent overfitting by turning of 25% of neurons
model.add(Dense(25, activation='softmax'))  # final output layer. classifies 25 letters.
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 128)    │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 14, 14, 64)     │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 7, 7, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 25)             │        12,825 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 319,865 (1.22 MB)

 Trainable params: 319,865 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

## 4. Model Training

In [37]:
# Compile model using automatic learning rate adjustment for multi-class classification

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
train_datagen = ImageDataGenerator(
    rotation_range=0,
    height_shift_range=0.2,
    width_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

model.fit(
    train_datagen.flow(X_train, Y_train, batch_size=200),
    epochs=35,
    validation_data=(X_test, Y_test),
    shuffle=1
)

Epoch 1/35


/Users/adityasharma/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.0615 - loss: 3.1473 - val_accuracy: 0.2591 - val_loss: 2.4433
Epoch 2/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.2273 - loss: 2.5096 - val_accuracy: 0.4969 - val_loss: 1.5065
Epoch 3/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.4161 - loss: 1.7952 - val_accuracy: 0.6407 - val_loss: 1.0783
Epoch 4/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - accuracy: 0.5506 - loss: 1.3527 - val_accuracy: 0.7002 - val_loss: 0.8459
Epoch 5/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.6295 - loss: 1.1026 - val_accuracy: 0.8179 - val_loss: 0.5501
Epoch 6/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.6934 - loss: 0.9068 - val_accuracy: 0.8072 - val_loss: 0.5429
Epoch 7/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.7393 - loss: 0.7749 - val_accuracy: 0.8617 - val_loss: 0.4291
Epoch 8/35
138/138 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - accuracy: 0.7751 - loss: 0.6653 - val_accuracy: 0.871

## 5. Final Model Metrics

In [39]:
loss, accuracy = model.evaluate(X_test, Y_test)

225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9983 - loss: 0.0137


In [40]:
print(f"Test accuracy: {accuracy * 100}%")
print(f"Final loss: {loss}")

Test accuracy: 99.87451434135437%
Final loss: 0.013428001664578915


In [41]:
model.save("../models/model.keras")